In [14]:
!pip install keras-tuner


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.2 MB/s eta 0:00:00


In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [3]:
tf.__version__

'2.11.0'

In [6]:
fashion_mnist=keras.datasets.fashion_mnist

In [7]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [8]:
train_images=train_images/255.0
test_images=test_images/255.0

In [9]:
train_images.shape

(60000, 28, 28)

In [10]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [15]:
def build_model(hp):
  model=keras.Sequential([
  keras.layers.Conv2D(
      filters=hp.Int("conv_1_filter",min_value=32,max_value=128,step=16),
      kernel_size=hp.Choice("conv_1_kernel",values=[3,5]),
      activation ="relu",
      input_shape=[28,28,1]
      ),
  keras.layers.Conv2D(
      filters=hp.Int("conv_2_filter",min_value=32,max_value=64,step=16),
      kernel_size=hp.Choice("conv_2_kernel",values=[3,5]),
      activation ="relu"
      ),
  keras.layers.Flatten(),
  keras.layers.Dense(
      units=hp.Int("dense_1_units",min_value=32,max_value=128,step=16),
      activation="relu"
      ),
  keras.layers.Dense(10,activation="softmax") 
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate",values=[1e-2,1e-3])),
                 loss="sparse_categorical_crossentropy",
                 metrics=["accuracy"])
  
  return model



   





In [16]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-16-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [17]:
tuner_search=RandomSearch(build_model,objective="val_accuracy",max_trials=5,directory="output",project_name="Mnist_Fashion")
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 30s]
val_accuracy: 0.9043333530426025

Best val_accuracy So Far: 0.9168333411216736
Total elapsed time: 00h 02m 53s


In [18]:
model=tuner_search.get_best_models(num_models=1)[0]

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 48)        13872     
                                                                 
 flatten (Flatten)           (None, 27648)             0         
                                                                 
 dense (Dense)               (None, 96)                2654304   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 2,669,466
Trainable params: 2,669,466
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.1153 - accuracy: 0.9575 - val_loss: 0.2736 - val_accuracy: 0.9177
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0774 - accuracy: 0.9716 - val_loss: 0.2746 - val_accuracy: 0.9170
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0530 - accuracy: 0.9804 - val_loss: 0.3587 - val_accuracy: 0.9137
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0369 - accuracy: 0.9869 - val_loss: 0.4170 - val_accuracy: 0.9132
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0281 - accuracy: 0.9899 - val_loss: 0.4450 - val_accuracy: 0.9147
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0221 - accuracy: 0.9921 - val_loss: 0.5392 - val_accuracy: 0.9158
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0190 - accuracy: 0.9933 - val_loss: 0.5274 - val_accurac